In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
title = pd.read_csv('FN_filtered_syn_title.csv')

In [3]:
title

,title,url,year,tconst,isAdult,genres,synopsis
0,Logan,/release/rl3229320705/?ref_=bo_rs_table_4,2017,tt3315342,0,"Action,Drama,Sci-Fi",The theatrical release of Logan was preceded b...
1,The Shack,/release/rl2642183681/?ref_=bo_rs_table_5,2017,tt2872518,0,"Drama,Fantasy","A 13-year-old boy named Mackenzie ""Mack"" Phill..."
2,Before I Fall,/release/rl4001269249/?ref_=bo_rs_table_6,2017,tt1691916,0,"Drama,Fantasy,Mystery",Samantha Kingston wakes on at 6:50 am on Febru...
3,Table 19,/release/rl947815937/?ref_=bo_rs_table_7,2017,tt1412528,0,"Comedy,Drama,Romance",Eloise McGarry attends the wedding of her olde...
4,Kong: Skull Island,/release/rl710837761/?ref_=bo_rs_table_21,2017,tt3731562,0,"Action,Adventure,Fantasy","In the year 1944, somewhere over the South Pac..."
...,...,...,...,...,...,...,...
332,Blinded by the Light,/release/rl1409910273/?ref_=bo_rs_table_48,2019,tt8266310,0,"Comedy,Drama,Music","n 1980, young Javed Khan is with his best fri..."
333,Ready or Not,/release/rl285771265/?ref_=bo_rs_table_61,2019,tt7798634,0,"Action,Comedy,Horror","Opening in the year 1989, inside the Le Domas ..."
334,Angel Has Fallen,/release/rl1745257985/?ref_=bo_rs_table_64,2019,tt6189022,0,"Action,Thriller",The film starts with Mike Banning undergoing ...
335,Overcomer,/release/rl3808724481/?ref_=bo_rs_table_65,2019,tt8186318,0,"Drama,Family,Sport",The opening scene takes us into the gym of Bro...


In [4]:
def movie_id(tid):
    url = f'https://www.imdb.com/title/{tid}/reviews?ref_=tturv_ql_3'
    base_url = "https://www.imdb.com/"
    req=requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    load_more = soup.select(".load-more-data")
    
    flag = True
    if load_more[0].select(".ipl-load-more__load-indicator"):
        ajaxurl = load_more[0]['data-ajaxurl']
        base_url = base_url + ajaxurl + "?ref_=undefined&paginationKey="
        key = load_more[0]['data-key']
    else:
        flag = False
    
    title_list = []
    id_list = []
    date_list = []
    rating_list = []
    review_list = []
    spoiler_list = []
    help_list = []
    
    df = pd.DataFrame()
    #수집하고 싶은 영화 리뷰 수 지정
    MAX_CNT = 200
    cnt = 0

    while flag:
        url = base_url + key
        req=requests.get(url)
        soup = BeautifulSoup(req.text, "lxml")
        item = soup.select(".lister-list")
        for item in soup.select(".lister-list"):
            for i in range(len(item.select('a.title'))):        
                title = tid
                rid = item.select('a.title')[i]['href']
                date = item.select("span.review-date")[i].text
                if len(item.select(".lister-item-content")[i].select(".rating-other-user-rating")) != 0:
                    rating = item.select(".lister-item-content")[i].select(".rating-other-user-rating > span")[0::2][0].text
                else:
                    rating = "no rating"
                review = item.select(".text")[i].text
                spoiler = item.select(".expander-icon-wrapper")[i]['class'][1]
                helpful = item.find_all("div", "actions text-muted")[i].contents[0]

                title_list.append(title)
                id_list.append(rid)
                date_list.append(date)
                rating_list.append(rating)
                review_list.append(review)
                spoiler_list.append(spoiler)
                help_list.append(helpful)
            cnt = cnt + 1

        if cnt >= MAX_CNT:
            break
        load_more = soup.select(".load-more-data")
        if len(load_more):
            key = load_more[0]['data-key']
        else:
            flag = False
        
    df['title'] = title_list
    df['id'] = id_list
    df['date'] = date_list
    df['review'] = review_list
    df['rating'] = rating_list
    df['spoiler'] = spoiler_list
    df['helpful'] = help_list
    
    return df

In [5]:
tids = range(title.shape[0])

appended_data = []
for idx, value in enumerate(tids):
    tid_list = title.iloc[value,3]
    review = movie_id(tid_list)
    appended_data.append(review)
    
review_df = pd.concat(appended_data)
review_df

,title,id,date,review,rating,spoiler,helpful
0,tt3315342,/review/rw3664084/,17 March 2017,I really can't get how in the world people are...,6,spoiler-warning__control,\n 330 out of 684 found thi...
1,tt3315342,/review/rw3780409/,14 August 2017,With a bit more depth and dramatic flair then ...,8,show-more__control,\n 39 out of 65 found this ...
2,tt3315342,/review/rw3764270/,26 July 2017,LOGAN is a fitting follow-up to Hugh Jackman's...,7,spoiler-warning__control,\n 25 out of 52 found this ...
3,tt3315342,/review/rw4582928/,14 January 2019,"I've seen several X-men films already, but I h...",9,spoiler-warning__control,\n 13 out of 24 found this ...
4,tt3315342,/review/rw4000491/,13 December 2017,Nearly everyone I've ever come across unanimou...,5,spoiler-warning__control,\n 15 out of 24 found this ...
...,...,...,...,...,...,...,...
196,tt6803212,/review/rw7324125/,9 September 2021,This movie deserves a 10 for the outstanding p...,10,show-more__control,\n 0 out of 3 found this he...
197,tt6803212,/review/rw7028653/,14 June 2021,Very good movie. But you need to get a lot att...,10,spoiler-warning__control,\n 0 out of 1 found this he...
198,tt6803212,/review/rw7117639/,12 July 2021,This movie was absolutely fantastic. It had me...,10,show-more__control,\n 0 out of 1 found this he...
199,tt6803212,/review/rw7217551/,11 August 2021,Don't listen to the bad reviews! Just wow! Thi...,10,show-more__control,\n 0 out of 3 found this he...


In [6]:
review_df.to_csv('FN_review.csv', index=False)